In [87]:
import pandas as pd
import numpy as np
import requests
import ujson
import os

import plotly
from plotly import tools

import plotly
with open('/Users/timlee/Dropbox/keys/plotly_apikey.txt','r') as f:
    api_key = f.read()
plotly.tools.set_credentials_file(username='tdlee', api_key=api_key)

import plotly.plotly as ply
import plotly.graph_objs as go
from plotly.grid_objs import Grid, Column

In [88]:
country_codes = ['chn', 'idn', 'ind', 'jpn', 'sau', 'tur', 'twn',
                 'bel', 'che', 'deu', 'esp', 'fra', 'gbr', 'ita',
                 'nld', 'pol', 'rus', 'swe', 'can', 'mex', 'usa',
                 'aus', 'arg', 'bra']


def get_countries_lkup():
    countries_url = 'https://atlas.media.mit.edu/attr/country/'
    resp = requests.get(countries_url)
    resp_json = ujson.loads(resp.content)['data']
    resp_df = pd.DataFrame(resp_json)
    country_id_lkup = {idx: name for idx, name in resp_df[['display_id','name']].values}
    return country_id_lkup


def get_data(countrycode):
    imports_url = './OEC_%s_imports.feather' % countrycode
    exports_url = './OEC_%s_exports.feather' % countrycode
    
    if not os.path.isfile(imports_url):
        return None, None
    
    if not os.path.isfile(exports_url):
        return None, None
        
    
    df_exports = pd.read_feather(exports_url)
    df_exports = df_exports[df_exports['export_val'].isna()==False].copy()
    df_exports = df_exports.sort_values(by='export_val')
    
    df_imports = pd.read_feather(imports_url)
    df_imports = df_imports[df_imports['import_val'].isna() == False].copy()
    df_imports = df_imports.sort_values(by = 'import_val')
    
    return df_imports, df_exports


def topbot_impexp(df_imports, df_exports, n=7):
    mask = df_imports['year'] == 2014
    tmp_imp = df_imports.loc[mask].sort_values(by='import_val')
    mask = df_exports['year'] == 2014
    tmp_exp = df_exports.loc[mask].sort_values(by='export_val')
    
    top_imp_prod = tmp_imp.tail(n)['product_name'].values
    top_exp_prod = tmp_exp.tail(n)['product_name'].values
    
    imp_top_traces = []
    for prod in top_imp_prod[::-1]:
        mask = df_imports['product_name'] == prod
        tmp = df_imports.loc[mask,['year','import_val']].copy()
        tmp = tmp.groupby('year')['import_val'].sum().reset_index()
        tr = go.Scatter(
            x = tmp['year'].values,
            y = tmp['import_val'].values,
            mode = 'lines',
            line = dict(width=6),
            name = prod
        )
        imp_top_traces.append(tr)
    
    exp_top_traces = []
    for prod in top_exp_prod[::-1]:
        mask = df_exports['product_name'] == prod
        tmp = df_exports.loc[mask,['year','import_val']].copy()
        tmp = tmp.groupby('year')['import_val'].sum().reset_index()
        tr = go.Scatter(
            x = tmp['year'].values,
            y = tmp['import_val'].values,
            mode = 'lines',
            line = dict(width=6),
            name = prod
        )
        exp_top_traces.append(tr)
    
    return imp_top_traces, exp_top_traces


def gen_all_traces(country_codes):
    imp_trs = []
    exp_trs = []
    actual_ccs = []
    
    for countrycode in country_codes:
        df_imports, df_exports = get_data(countrycode)
        if df_imports is not None:
            actual_ccs.append(countrycode)
            imp_top_traces, exp_top_traces = topbot_impexp(df_imports, df_exports)
            imp_trs.extend(imp_top_traces)
            exp_trs.extend(exp_top_traces)
    
    return imp_trs, exp_trs, actual_ccs


def get_updatemenus(actual_ccs, country_id_lkup):
    filters = {}
    buttons = []

    for idx, countrycode in enumerate(actual_ccs):
        template = np.repeat(False, 154)
        curr_idx = idx*7
        template[curr_idx:curr_idx+7] = True
        filters[countrycode] = template

        buttons.append(dict(label=country_id_lkup[countrycode],
                            method='update',
                            args=[{
                                'visible': template,
                                'title': countrycode
                            }]
                           ))
    updatemenus = list([
        dict(
             direction='down',
             active=2,
             buttons=buttons,
             x = 0,
             xanchor = 'left',
             y = 1.1,
             yanchor = 'top',
             font={'size':12},
             showactive = True,
            )
    ])
    return updatemenus


imp_trs, exp_trs, actual_ccs = gen_all_traces(country_codes)
country_id_lkup = get_countries_lkup()
updatemenus = get_updatemenus(actual_ccs, country_id_lkup)

In [91]:
font_dict = dict(
            family='Roboto',
            size=20
        )

layout = go.Layout(
    font = font_dict,
    hovermode = 'closest',
    width=900,
    height=600,
    xaxis=dict(
        title='Year',
        titlefont=font_dict,
        tickfont=font_dict,        
    ),
    yaxis=dict(
        title='Tonnes',
        titlefont=font_dict,
        tickfont=font_dict,        
    ),
    legend=dict(
        x=0,
        y=1,
        traceorder='normal',
        font=dict(
            family='Roboto',
            size=12,
            color='#000'
        ),
        bgcolor='#E2E2E2',
        bordercolor='#FFFFFF',
        borderwidth=2
    ),
    updatemenus=updatemenus
)

layout.update(title="Top Imports by Country")
fig = dict(data=imp_trs, layout=layout)
ply.iplot(fig, filename='TopImports_DropDown')

In [92]:
layout.update(title="Top Exports by Country")
fig = dict(data=exp_trs, layout=layout)
ply.iplot(fig, filename='TopExports_DropDown')

In [81]:
ply.iplot(exp_top_traces)